In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

%cd /content/drive/MyDrive/fyp

/content/drive/.shortcut-targets-by-id/1nxIH2xjXasApCEuy6EcHF76-LsPHN_-u/fyp


In [ ]:
#real time data capture:

In [ ]:
#below function is to run cam on google collab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/fyp

from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
import time

# Function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
    """
    Params:
        js_reply: JavaScript object containing an image from the webcam
    Returns:
        img: OpenCV BGR image
    """
    # Decode base64 image
    image_bytes = b64decode(js_reply.split(',')[1])
    # Convert bytes to numpy array
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    # Decode numpy array into OpenCV BGR image
    img = cv2.imdecode(jpg_as_np, flags=1)

    return img

def take_photo(filename='/content/drive/MyDrive/fyp/input_img.jpg', quality=10):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({ video: true });

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Wait for Capture to be clicked.
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)

    # Get photo data
    data = eval_js('takePhoto({})'.format(quality))
    # Get OpenCV format image
    img = js_to_image(data)

    # Save the image
    cv2.imwrite(filename, img)

    try:
        # Wait for a moment to ensure the image is saved
        time.sleep(1)
        # Show the image that was just taken
        display(Image(filename))
    except Exception as err:
        print(str(err))

try:
    filename = take_photo()
    print('Saved to {}'.format(filename))
except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1nxIH2xjXasApCEuy6EcHF76-LsPHN_-u/fyp


<IPython.core.display.Javascript object>

FileNotFoundError: ignored

FileNotFoundError: ignored

<IPython.core.display.Image object>

Saved to None


In [ ]:
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

<IPython.core.display.Javascript object>

Saved to input_img.jpg


FileNotFoundError: ignored

FileNotFoundError: ignored

<IPython.core.display.Image object>

In [ ]:
#below function is to run cam on jupyter and other python ide

#import cv2
from google.colab.patches import cv2_imshow

#from google.colab.patches import cv2_imshow
#cv2_imshow(img)

webcam = cv2.VideoCapture(0)

while(True):
    ret, frame = webcam.read()
    cv2_inshow(frame)
    #cv2_imshow("output",frame)
    cv2_waitKey(1)

    if cv2.waitKey(100) & 0xFF == ord('S'):
        cv2.imwrite("input.jpg", frame)
        cv2.destroyAllWindows()
        break
    if cv2.waitKey(100) & 0xFF == ord('Q'):
        cv2.destroyAllWindows()
        break
webcam.release()

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
!curl -o logo.png https://colab.research.google.com/img/colab_favicon_256px.png
import cv2
img = cv2.imread('logo.png', cv2.IMREAD_UNCHANGED)
cv2_imshow(img)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: Failed to create the file logo.png: Read-only file system
 30  4534   30  1377    0     0  12347      0 --:--:-- --:--:-- --:--:-- 12405
curl: (23) Failure writing output to destination


AttributeError: ignored

In [ ]:
#Feature Extraction using YOLO:
#checks presence of person and extracts largest boundary box which has person

In [ ]:
import torch
# Model
!pip install ultralytics
from ultralytics import YOLO
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

In [ ]:

imgpred=['input_img.jpg']
res = model(imgpred)
res


In [ ]:
pred = res.pandas().xyxy[0]
pred

In [ ]:
if 0 in pred['class'].unique():
    print('Person is detected')
else:
    print('No person detected')

In [ ]:
itr = 0
ind = 0
area = 0
for index, row in pred.iterrows():
    if str(row['class'])!="0":
        continue
    if area < ((int(row['xmax'])-int(row['xmin']))*(int(row['ymax'])-int(row['ymin']))):
        ind = itr
        area = ((int(row['xmax'])-int(row['xmin']))*(int(row['ymax'])-int(row['ymin'])))
    itr+=1


In [ ]:

bbox_raq = res.xyxy[0][ind]
bbox = []
for bound in bbox_raq:
    bbox.append(int(bound.item()))
bbox

In [ ]:
import cv2
from matplotlib import pyplot as plt
image = cv2.imread('input.jpg')
img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
crp = img_rgb[bbox[1]:bbox[3], bbox[0]:bbox[2]]


In [ ]:
import numpy as np
from PIL import Image as im

data = im.fromarray(crp)
data.save('input_images/cr_img.jpg')

In [ ]:
#Outfit Segmentation using u2net:


In [ ]:
import os
from tqdm import tqdm
from tqdm.notebook import tqdm
import numpy as np
from PIL import Image as im
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms

from data.normalize import Normalize_image
from util.saving_utils import load_checkpoint_mgpu

from networks import U2NET
device = 'cpu'

image_dir = 'input_images'
result_dir = 'output_images'
checkpoint_path = 'cloth_segm_u2net_latest.pth'

transforms_list = []
transforms_list += [transforms.ToTensor()]
transforms_list += [Normalize_image(0.5, 0.5)]
transform_rgb = transforms.Compose(transforms_list)

net = U2NET(in_ch=3, out_ch=4)
net = load_checkpoint_mgpu(net, checkpoint_path)
net = net.to(device)
net = net.eval()



#!del -rf input_images/.ipynb_checkpoints
image_name = sorted(os.listdir(image_dir))[0]

img = im.open(os.path.join(image_dir, image_name)).convert('RGB')
l,h= img.size
img_size = img.size
img = img.resize((768, 768), im.BICUBIC)
image_tensor = transform_rgb(img)
image_tensor = torch.unsqueeze(image_tensor, 0)

output_tensor = net(image_tensor)
output_tensor = F.log_softmax(output_tensor[0], dim=1)
output_tensor = torch.max(output_tensor, dim=1, keepdim=True)[1]
output_tensor = torch.squeeze(output_tensor, dim=0)
output_tensor = torch.squeeze(output_tensor, dim=0)
output_arr = output_tensor.cpu().numpy()

output_img = im.fromarray(output_arr.astype('uint8'), mode='L')
output_img = output_img.resize(img_size, im.BICUBIC)


In [ ]:
#bottom_outfit

palette = [0,0,0,0,0,0,225,225,225,0,0,0]

mask= im.fromarray(output_arr.astype('uint8'),mode='L')
mask.putpalette(palette)
mask.save(os.path.join(result_dir, 'mask_bottom.png'))


In [ ]:
import cv2
import numpy as np

img = cv2.imread('input_images/cr_img.jpg')


np.shape(img)
x = cv2.imread("output_images/mask_bottom.png",0)
x=cv2.resize(x,(l,h))
x=x.astype("uint8")
img=img.astype("uint8")
np.shape(x)

res = cv2.bitwise_and(img,img,mask=x)
image = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
image = im.fromarray(image)
image.save(os.path.join(result_dir,'bottom.png'))

In [ ]:

#top_outfit
palette = [0,0,0,225,225,225,0,0,0,0,0,0]

mask= im.fromarray(output_arr.astype('uint8'),mode='L')

mask.putpalette(palette)
mask.save(os.path.join(result_dir, 'top_mask.png'))


In [ ]:
import cv2
import numpy as np

img = cv2.imread('input_images/cr_img.jpg')

np.shape(img)
x = cv2.imread("output_images/top_mask.png",0)
x=cv2.resize(x,(l,h))
x=x.astype("uint8")
img=img.astype("uint8")
np.shape(x)

res = cv2.bitwise_and(img,img,mask=x)
image = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
image = im.fromarray(image)
image.save(os.path.join(result_dir,'top.png'))

In [ ]:


#fullbody_outfit
palette = [0,0,0,0,0,0,0,0,0,225,225,225]

mask= im.fromarray(output_arr.astype('uint8'),mode='L')

mask.putpalette(palette)
mask.save(os.path.join(result_dir, 'fullbody_mask.png'))


In [ ]:
import cv2
import numpy as np

img = cv2.imread('input_images/cr_img.jpg')

#img=cv2.resize(img,(768,768))

np.shape(img)
x = cv2.imread("output_images/fullbody_mask.png",0)
x=cv2.resize(x,(l,h))
x=x.astype("uint8")
img=img.astype("uint8")
np.shape(x)

res = cv2.bitwise_and(img,img,mask=x)
image = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
image = im.fromarray(image)
image.save(os.path.join(result_dir,'fullbody.png'))

In [ ]:
#to check if fullbody image is present or not
fb = cv2.imread("output_images/fullbody.png")
fb=cv2.resize(fb,(500,500))

black = np.zeros((500,500,3),dtype=np.uint8) #make black color image
black = 255*black

isfull_body__not_present= np.array_equal(black,fb) #compare those two images if returns true then full body cloth is not present

isfull_body__not_present

In [ ]:
if isfull_body__not_present==0:
    x = cv2.imread("output_images/fullbody_mask.png",0)
    y = cv2.imread("output_images/top_mask.png",0)
    new_mask = cv2.bitwise_or(x,y)
    image = cv2.cvtColor(new_mask, cv2.COLOR_BGR2RGB)
    image = im.fromarray(image)
    image.save(os.path.join(result_dir,'fullbody_mask.png'))
    image.save(os.path.join(result_dir,'top_mask.png'))

    #full_body:
    img = cv2.imread('input_images/cr_img.jpg')

    #img=cv2.resize(img,(768,768))

    x = cv2.imread("output_images/fullbody_mask.png",0)
    x=cv2.resize(x,(l,h))
    x=x.astype("uint8")
    img=img.astype("uint8")
    np.shape(x)

    res = cv2.bitwise_and(img,img,mask=x)
    image = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
    image = im.fromarray(image)
    image.save(os.path.join(result_dir,'fullbody.png'))

    #top:
    img = cv2.imread('input_images/cr_img.jpg')

    x = cv2.imread("output_images/top_mask.png",0)
    x=cv2.resize(x,(l,h))
    x=x.astype("uint8")
    img=img.astype("uint8")
    np.shape(x)

    res = cv2.bitwise_and(img,img,mask=x)
    image = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
    image = im.fromarray(image)
    image.save(os.path.join(result_dir,'top.png'))

In [ ]:
pip install rembg

In [ ]:

#create white background to be applied upon top , bottom and full body outfits
import cv2
import numpy as np
import os
from rembg import remove
from PIL import Image
white = np.ones((h,l,3),dtype=np.uint8)
white = 255*white

w = cv2.cvtColor(white, cv2.COLOR_BGR2RGB)
image = Image.fromarray(w)
image.save(os.path.join('white_background_for_outfit.png'))

In [ ]:
#top image
input = Image.open('output_images/top.png')
mask = remove(input)
bg = Image.open('white_background_for_outfit.png')
outpt = Image.composite(input,bg,mask)
outpt.save('output_images/top.png')

In [ ]:
#bottom image
input = Image.open('output_images/bottom.png')
mask = remove(input)
bg = Image.open('white_background_for_outfit.png')
outpt = Image.composite(input,bg,mask)
outpt.save('output_images/bottom.png')

In [ ]:
#this function is to crop top half of the bottom.png to cut out the white space
img = Image.open('output_images/bottom.png')

x,y = img.size

img=img.crop((0,y/2,x,y))
img.save('output_images/bottom.png')

In [ ]:
#Full body
if not isfull_body__not_present:

    input = Image.open('output_images/full_body.png')
    mask = remove(input)
    bg = Image.open('white_background_for_outfit.png')
    outpt = Image.composite(input,bg,mask)
    outpt.save('output_images/full_body.png')
else:
    print("Full body outfit is not present")

In [ ]:
#change hue of the image

In [ ]:
from torchvision.io import read_image
import torchvision.transforms.functional as F
import torchvision.transforms as T

In [ ]:
#top
img = read_image('output_images/top.png')
img1 =  F.adjust_hue(img, -0.4)
img2 =  F.adjust_hue(img, 0.2)

pimg1 = T.ToPILImage()(img1)
pimg2 = T.ToPILImage()(img2)

pimg1.save('output_images/top_1.png')
pimg2.save('output_images/top_2.png')

In [ ]:
#bottom
img = read_image('output_images/bottom.png')
img1 =  F.adjust_hue(img, -0.4)
img2 =  F.adjust_hue(img, 0.2)

pimg1 = T.ToPILImage()(img1)
pimg2 = T.ToPILImage()(img2)

pimg1.save('output_images/bottom_1.png')
pimg2.save('output_images/bottom_2.png')

In [ ]:
#full_body
img = read_image('output_images/fullbody.png')
img1 =  F.adjust_hue(img, -0.4)
img2 =  F.adjust_hue(img, 0.2)

pimg1 = T.ToPILImage()(img1)
pimg2 = T.ToPILImage()(img2)

pimg1.save('output_images/fb_1.png')
pimg2.save('output_images/fb_2.png')

In [ ]:
#below code snippet takes the input of live capture module and removes the bacckground and replaces it with white background
#the output image should be given as input image for AGCPN Virtual Try on

In [ ]:
img = im.open('input.jpg')
l,h=input.size
white = np.ones((h,l,3),dtype=np.uint8)
white = 255*white

w = cv2.cvtColor(white, cv2.COLOR_BGR2RGB)
image = Image.fromarray(w)
image.save(os.path.join('white_background_for_input_image.png'))

In [ ]:
from rembg import remove
from PIL import Image
input = Image.open('input.jpg')
mask = remove(input)
bg = Image.open('white_background_for_input_image.png')
outpt = Image.composite(input,bg,mask)
outpt.save('INPUT_FOR_VIRTUAL_TRY_ON.png')

In [ ]:
#Now age and Gender classification
!pip install DeepFace

In [ ]:
from deepface import DeepFace

In [ ]:
objs = DeepFace.analyze(img_path = "input_images/cr_img.jpg",
        actions = ['age', 'gender', 'race']
)

In [ ]:
objs

In [ ]:
Age=objs[0]['age']
Gender = objs[0]['dominant_gender']

In [ ]:

if Age<=12:
    Age_cat='child'
elif Age<=19:
    Age_cat='Teen'
else:
    Age_cat='Adult'

Age_cat

In [ ]:

if Gender == 'Man':
    Gender_cat='Male'
else:
    Gender_cat='Female'
Gender_cat

In [ ]:
#Outfit_Classification

In [ ]:
%cd outfit_classification
!pip install pretrainedmodels

In [ ]:
#declare the resnet-50 model
import torch.nn as nn
import torch.nn.functional as F
import pretrainedmodels

class MultiHeadResNet50(nn.Module):
    def __init__(self):
        super(MultiHeadResNet50, self).__init__()
        self.model = pretrainedmodels.__dict__['resnet50'](pretrained=None)

        for param in self.model.parameters():
            param.requires_grad = False
        print('Freezing intermediate layer parameters...')

        # change the final layer
        self.l0 = nn.Linear(2048, 8)
        self.l1 = nn.Linear(2048, 57)

    def forward(self, x):
        # get the batch size only, ignore (c, h, w)
        batch, _, _, _ = x.shape
        x = self.model.features(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch, -1)
        l0 = self.l0(x)
        l1 = self.l1(x)
        return l0, l1

In [ ]:
#load the model
import torch
import cv2
import torchvision.transforms as transforms
import numpy as np
import joblib

import urllib.request

device = torch.device('cpu')
model = MultiHeadResNet50()
checkpoint = torch.load('model.pth', map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [ ]:

#classification for top outfit
image = cv2.imread('../output_images/top.png')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# apply image transforms
image = transform(image)
# add batch dimension
image = image.unsqueeze(0).to(device)
# forward pass the image through the model
outputs = model(image)
# extract the three output
output1, output2 = outputs
# get the index positions of the highest label score
out_label_1 = np.argmax(output1.detach().cpu())
out_label_2 = np.argmax(output2.detach().cpu())
num_list_art = joblib.load('./num_list_art.pkl')
num_list_sub = joblib.load('./num_list_sub.pkl')
# get the keys and values of each label dictionary
art_keys = list(num_list_art.keys())
art_values = list(num_list_art.values())
sub_keys = list(num_list_sub.keys())
sub_values = list(num_list_sub.values())
final_labels = []
# append the labels by mapping the index position to the values
final_labels.append(art_keys[out_label_2])
final_labels.append(sub_keys[out_label_1])
print('article_type:'+final_labels[0])
print('sub_cat_type:'+final_labels[1])

Article_type_Top = final_labels[0]
Category_type_Top = final_labels[0]

In [ ]:

#classification for bottom outfit
image = cv2.imread('../output_images/bottom.png')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# apply image transforms
image = transform(image)
# add batch dimension
image = image.unsqueeze(0).to(device)
# forward pass the image through the model
outputs = model(image)
# extract the three output
output1, output2 = outputs
# get the index positions of the highest label score
out_label_1 = np.argmax(output1.detach().cpu())
out_label_2 = np.argmax(output2.detach().cpu())
num_list_art = joblib.load('./num_list_art.pkl')
num_list_sub = joblib.load('./num_list_sub.pkl')
# get the keys and values of each label dictionary
art_keys = list(num_list_art.keys())
art_values = list(num_list_art.values())
sub_keys = list(num_list_sub.keys())
sub_values = list(num_list_sub.values())
final_labels = []
# append the labels by mapping the index position to the values
final_labels.append(art_keys[out_label_2])
final_labels.append(sub_keys[out_label_1])
print('article_type:'+final_labels[0])
print('subcat:'+final_labels[1])
Article_type_bottom = final_labels[0]
Category_type_bottom = final_labels[0]

In [ ]:
#@title
#classification for full_body outfit
if not isfull_body__not_present:

    image = cv2.imread('../output_images/fullbody.png')

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # apply image transforms
    image = transform(image)
    # add batch dimension
    image = image.unsqueeze(0).to(device)
    # forward pass the image through the model
    outputs = model(image)
    # extract the three output
    output1, output2 = outputs
    # get the index positions of the highest label score
    out_label_1 = np.argmax(output1.detach().cpu())
    out_label_2 = np.argmax(output2.detach().cpu())
    num_list_art = joblib.load('./num_list_art.pkl')
    num_list_sub = joblib.load('./num_list_sub.pkl')
    # get the keys and values of each label dictionary
    art_keys = list(num_list_art.keys())
    art_values = list(num_list_art.values())
    sub_keys = list(num_list_sub.keys())
    sub_values = list(num_list_sub.values())
    final_labels = []
    # append the labels by mapping the index position to the values
    final_labels.append(art_keys[out_label_2])
    final_labels.append(sub_keys[out_label_1])
    print('article_type:'+final_labels[0])
    print('subcat:'+final_labels[1])


    Article_type_fullbody = final_labels[0]
    Category_type_fullbody = final_labels[0]
else:
    print("Full-Body outfit not present")

In [ ]:

#Recommendation System
%cd ..

In [ ]:
#@title
pip install -U numpy

In [ ]:
import pickle
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.models import Sequential
from numpy.linalg import norm
from sklearn.neighbors import NearestNeighbors
import cv2
import os

In [ ]:
#Recommends top outfit and saves the images in recom-top folder

In [ ]:
# Feature_embedding = "image_features_embedding_{}_{}_{}.pkl".format(Gender_cat,Age_cat,Article_Type_Top/bottom/fullbody)
# Files_name = "img_files_{}_{}_{}.pkl".format(Gender_cat,Age_cat,Article_Type_Top/bottom/fullbody)

Feature_embedding = "image_features_embedding_top_{}.pkl".format(Gender_cat)
Files_name = "img_files_top_{}.pkl".format(Gender_cat)

In [ ]:
features_list = pickle.load(open(Feature_embedding, "rb"))
img_files_list = pickle.load(open(Files_name, "rb"))



model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
model.trainable = False
i=0
print("Image file names of the recommended outfit:")
model = Sequential([model, GlobalMaxPooling2D()])
#0
img = image.load_img('output_images/top.png',target_size=(224,224))
img_array = image.img_to_array(img)
expand_img = np.expand_dims(img_array,axis=0)
preprocessed_img = preprocess_input(expand_img)
result_to_resnet = model.predict(preprocessed_img)
flatten_result = result_to_resnet.flatten()
# normalizing
result_normlized = flatten_result / norm(flatten_result)


neighbors = NearestNeighbors(n_neighbors = 6, algorithm='brute', metric='euclidean')
neighbors.fit(features_list)

distence, indices = neighbors.kneighbors([result_normlized])

for file in indices[0][0:1]:
    print("{}.jpg".format(img_files_list[file]))
    tmp_img = cv2.imread(os.path.join("Inventory-data",str(img_files_list[file])+".jpg"),cv2.COLOR_RGB2BGR)

    cv2.imwrite(os.path.join("recom-top", str(i)+'reco.png'),tmp_img)
    i+=1

#1
img = image.load_img('output_images/top_1.png',target_size=(224,224))
img_array = image.img_to_array(img)
expand_img = np.expand_dims(img_array,axis=0)
preprocessed_img = preprocess_input(expand_img)
result_to_resnet = model.predict(preprocessed_img)
flatten_result = result_to_resnet.flatten()
# normalizing
result_normlized = flatten_result / norm(flatten_result)


neighbors = NearestNeighbors(n_neighbors = 6, algorithm='brute', metric='euclidean')
neighbors.fit(features_list)

distence, indices = neighbors.kneighbors([result_normlized])

for file in indices[0][0:1]:
    print("{}.jpg".format(img_files_list[file]))
    tmp_img = cv2.imread(os.path.join("Inventory-data",str(img_files_list[file])+".jpg"),cv2.COLOR_RGB2BGR)

    cv2.imwrite(os.path.join("recom-top", str(i)+'reco.png'),tmp_img)
    i+=1
#2
img = image.load_img('output_images/top_2.png',target_size=(224,224))
img_array = image.img_to_array(img)
expand_img = np.expand_dims(img_array,axis=0)
preprocessed_img = preprocess_input(expand_img)
result_to_resnet = model.predict(preprocessed_img)
flatten_result = result_to_resnet.flatten()
# normalizing
result_normlized = flatten_result / norm(flatten_result)


neighbors = NearestNeighbors(n_neighbors = 6, algorithm='brute', metric='euclidean')
neighbors.fit(features_list)

distence, indices = neighbors.kneighbors([result_normlized])

for file in indices[0][0:1]:
    print("{}.jpg".format(img_files_list[file]))
    tmp_img = cv2.imread(os.path.join("Inventory-data",str(img_files_list[file])+".jpg"),cv2.COLOR_RGB2BGR)

    cv2.imwrite(os.path.join("recom-top", str(i)+'reco.png'),tmp_img)
    i+=1

In [ ]:
#Recommends bottom outfits and saves the images in recom-bottom folder

Feature_embedding = "image_features_embedding_bottom_{}.pkl".format(Gender_cat)
Files_name = "img_files_bottom_{}.pkl".format(Gender_cat)

In [ ]:
features_list = pickle.load(open(Feature_embedding, "rb"))
img_files_list = pickle.load(open(Files_name, "rb"))

model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
model.trainable = False

model = Sequential([model, GlobalMaxPooling2D()])
print("Image file names of the recommended outfit:")

i=0
#0
img = image.load_img('output_images/bottom.png',target_size=(224,224))
img_array = image.img_to_array(img)
expand_img = np.expand_dims(img_array,axis=0)
preprocessed_img = preprocess_input(expand_img)
result_to_resnet = model.predict(preprocessed_img)
flatten_result = result_to_resnet.flatten()
# normalizing
result_normlized = flatten_result / norm(flatten_result)


neighbors = NearestNeighbors(n_neighbors = 6, algorithm='brute', metric='euclidean')
neighbors.fit(features_list)

distence, indices = neighbors.kneighbors([result_normlized])

for file in indices[0][0:1]:
    print("{}.jpg".format(img_files_list[file]))
    tmp_img = cv2.imread(os.path.join("Inventory-data",str(img_files_list[file])+".jpg"),cv2.COLOR_RGB2BGR)

    cv2.imwrite(os.path.join("recom-bottom", str(i)+'reco.png'),tmp_img)
    i+=1

#1
img = image.load_img('output_images/bottom_1.png',target_size=(224,224))
img_array = image.img_to_array(img)
expand_img = np.expand_dims(img_array,axis=0)
preprocessed_img = preprocess_input(expand_img)
result_to_resnet = model.predict(preprocessed_img)
flatten_result = result_to_resnet.flatten()
# normalizing
result_normlized = flatten_result / norm(flatten_result)


neighbors = NearestNeighbors(n_neighbors = 6, algorithm='brute', metric='euclidean')
neighbors.fit(features_list)

distence, indices = neighbors.kneighbors([result_normlized])

for file in indices[0][0:1]:
    print("{}.jpg".format(img_files_list[file]))
    tmp_img = cv2.imread(os.path.join("Inventory-data",str(img_files_list[file])+".jpg"),cv2.COLOR_RGB2BGR)

    cv2.imwrite(os.path.join("recom-bottom", str(i)+'reco.png'),tmp_img)
    i+=1
#2
img = image.load_img('output_images/bottom_2.png',target_size=(224,224))
img_array = image.img_to_array(img)
expand_img = np.expand_dims(img_array,axis=0)
preprocessed_img = preprocess_input(expand_img)
result_to_resnet = model.predict(preprocessed_img)
flatten_result = result_to_resnet.flatten()
# normalizing
result_normlized = flatten_result / norm(flatten_result)


neighbors = NearestNeighbors(n_neighbors = 6, algorithm='brute', metric='euclidean')
neighbors.fit(features_list)

distence, indices = neighbors.kneighbors([result_normlized])

for file in indices[0][0:1]:
    print("{}.jpg".format(img_files_list[file]))
    tmp_img = cv2.imread(os.path.join("Inventory-data",str(img_files_list[file])+".jpg"),cv2.COLOR_RGB2BGR)

    cv2.imwrite(os.path.join("recom-bottom", str(i)+'reco.png'),tmp_img)
    i+=1

In [ ]:
#Recommends full-body outfits and saves the images in recom-fullbody folder

In [ ]:
if not isfull_body__not_present:
    features_list = pickle.load(open(Feature_embedding, "rb"))
    img_files_list = pickle.load(open(Files_name, "rb"))




    model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    model.trainable = False

    model = Sequential([model, GlobalMaxPooling2D()])

    img = image.load_img('output_images/fullbody.png',target_size=(224,224))
    img_array = image.img_to_array(img)
    expand_img = np.expand_dims(img_array,axis=0)
    preprocessed_img = preprocess_input(expand_img)
    result_to_resnet = model.predict(preprocessed_img)
    flatten_result = result_to_resnet.flatten()
    # normalizing
    result_normlized = flatten_result / norm(flatten_result)


    neighbors = NearestNeighbors(n_neighbors = 6, algorithm='brute', metric='euclidean')
    neighbors.fit(features_list)

    distence, indices = neighbors.kneighbors([result_normlized])

    print(indices)
    i=0
    for file in indices[0][0:3]:
        print("{}.jpg".format(img_files_list[file]))
        tmp_img = cv2.imread(os.path.join("Inventory-data",str(img_files_list[file])+".jpg"),cv2.COLOR_RGB2BGR)

        cv2.imwrite(os.path.join("recom-fullbody", str(i)+'reco.png'),tmp_img)
else:
    print("Full-Body image not present, so recommendations cannot be made")

THE TOP RECOMMENDED OUTFIT IS USED FOR VIRTUAL TRY-ON


In [ ]:
%cd ACGPN/

In [ ]:
!pip install ninja

In [ ]:
import numpy as np
from PIL import Image
import IPython
import os
import sys

from predict_pose import generate_pose_keypoints

In [ ]:
!rm -rf Data_preprocessing/test_color/*
!rm -rf Data_preprocessing/test_colormask/*
!rm -rf Data_preprocessing/test_edge/*
!rm -rf Data_preprocessing/test_img/*
!rm -rf Data_preprocessing/test_label/*
!rm -rf Data_preprocessing/test_mask/*
!rm -rf Data_preprocessing/test_pose/*
!rm -rf inputs/cloth/*
!rm -rf inputs/img/*
!rm -rf results/*

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
%cd U-2-Net
import u2net_load
import u2net_run
u2net = u2net_load.model(model_name = 'u2netp')
%cd ..

In [ ]:
from PIL import Image

u2net_run.infer(u2net, '', 'Data_preprocessing/test_edge')

In [ ]:
cloth_name = '000001_1.png'
cloth_path = '/content/drive/MyDrive/fyp/recom-top/2reco.png'
cloth = Image.open(cloth_path)
cloth = cloth.resize((192, 256), Image.BICUBIC).convert('RGB')
cloth.save(os.path.join('Data_preprocessing/test_color', cloth_name))

u2net_run.infer(u2net, 'Data_preprocessing/test_color', 'Data_preprocessing/test_edge')

In [ ]:
import time

start_time = time.time()
img_name = '000001_0.png'
img_path ='/content/drive/MyDrive/fyp/INPUT_FOR_VIRTUAL_TRY_ON.png'
img = Image.open(img_path)
img = img.resize((192,256), Image.BICUBIC)

img_path = os.path.join('Data_preprocessing/test_img', img_name)
img.save(img_path)
resize_time = time.time()
print('Resized image in {}s'.format(resize_time-start_time))

!python3 Self-Correction-Human-Parsing-for-ACGPN/simple_extractor.py --dataset 'lip' --model-restore 'lip_final.pth' --input-dir 'Data_preprocessing/test_img' --output-dir 'Data_preprocessing/test_label'
parse_time = time.time()
print('Parsing generated in {}s'.format(parse_time-resize_time))


In [ ]:

pose_path = os.path.join('Data_preprocessing/test_pose', img_name.replace('.png', '_keypoints.json'))
generate_pose_keypoints(img_path, pose_path)
pose_time = time.time()
print('Pose map generated in {}s'.format(pose_time-parse_time))

In [ ]:

with open('Data_preprocessing/test_pairs.txt','w') as f:
    f.write('000001_0.png 000001_1.png')

In [ ]:
!python test.py

In [ ]:
output_grid = np.concatenate([np.array(Image.open('Data_preprocessing/test_img/000001_0.png')),
                np.array(Image.open('Data_preprocessing/test_color/000001_1.png')),
                np.array(Image.open('results/test/try-on/000001_0.png'))], axis=1)
image_grid = Image.fromarray(output_grid)
image_grid

In [ ]:
%cd ..

In [ ]:
!git clone https://github.com/megvii-research/NAFNet
%cd NAFNet

In [ ]:
!pip install -r requirements.txt
!pip install --upgrade --no-cache-dir gdown
!python3 setup.py develop --no_cuda_ext

In [ ]:
import gdown
gdown.download('https://drive.google.com/uc?id=14Fht1QQJ2gMlk4N1ERCRuElg8JfjrWWR', "./experiments/pretrained_models/", quiet=False)

In [ ]:
import torch

from basicsr.models import create_model
from basicsr.utils import img2tensor as _img2tensor, tensor2img, imwrite
from basicsr.utils.options import parse
import numpy as np
import cv2
import matplotlib.pyplot as plt

def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img
def img2tensor(img, bgr2rgb=False, float32=True):
    img = img.astype(np.float32) / 255.
    return _img2tensor(img, bgr2rgb=bgr2rgb, float32=float32)

def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('NAFNet output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)

def single_image_inference(model, img, save_path):
      model.feed_data(data={'lq': img.unsqueeze(dim=0)})

      if model.opt['val'].get('grids', False):
          model.grids()

      model.test()

      if model.opt['val'].get('grids', False):
          model.grids_inverse()

      visuals = model.get_current_visuals()
      sr_img = tensor2img([visuals['result']])
      imwrite(sr_img, save_path)


In [ ]:
opt_path = 'options/test/SIDD/NAFNet-width64.yml'
opt = parse(opt_path, is_train=False)
opt['dist'] = False
NAFNet = create_model(opt)

In [ ]:
input_path = '/content/drive/MyDrive/fyp/ACGPN/results/test/try-on/000001_0.png'
output_path = 'demo_output/noisy-demo-0.png'

img_input = imread(input_path)
inp = img2tensor(img_input)
single_image_inference(NAFNet, inp, output_path)
img_output = imread(output_path)
display(img_input, img_output)

In [ ]:
import gdown
gdown.download('https://drive.google.com/uc?id=14D4V4raNYIOhETfcuuLI3bGLB-OYIv6X', "./experiments/pretrained_models/", quiet=False)

In [ ]:
opt_path = 'options/test/REDS/NAFNet-width64.yml'
opt = parse(opt_path, is_train=False)
opt['dist'] = False
NAFNet = create_model(opt)

In [ ]:
input_path = '/content/drive/MyDrive/fyp/ACGPN/results/test/try-on/000001_0.png'
output_path = 'demo_output/noisy-demo-0.png'

img_input = imread(input_path)
inp = img2tensor(img_input)
single_image_inference(NAFNet, inp, output_path)
img_output = imread(output_path)
display(img_input, img_output)